In [3]:
import numpy as np
import cv2

# Stage 1

In [4]:
s2 = cv2.imread("Images/2_colour.jpeg") #Scene with 2 objects
mask = cv2.imread("Images/masks/souvenirs_no_3_colour_mask_2_mask.png") #Target object mask

In [5]:
def ExtractObject(S2, ObjectMask):
    """Extracts an object from an image scene based on the mask used"""

    final_im = ObjectMask*S2
    final_im = cv2.bitwise_not(final_im)
    return final_im

In [6]:
def ApplyFilter(ExtractedObject, FilterIndex):
    """Applies convolution on the object using pre-defined image kernels"""

    if FilterIndex==0: #Apply no filter
        FilteredExObject = ExtractedObject
    elif FilterIndex==1:
        print("")
        #Will need to define some kernels and use them here
    elif FilterIndex==2:
        print("")
        #Will need to define some kernels and use them here
    elif FilterIndex==2:
        print("")
        #Will need to define some kernels and use them here
    return FilteredExObject

In [7]:
def ObjectBlender(S1, FilteredExObject):
    """Adds the filtered extracted object to the image in scene S1"""

    alpha = 0.5
    beta = 1.0 - alpha
    BlendingResult = cv2.addWeighted(S1, alpha, FilteredExObject, beta, 0.0)
    cv2.imshow('Blended', BlendingResult)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return BlendingResult

In [8]:
def CompareResult(BlendingResult, S2, metric):
    """Compares the blended image with the scene with 2 objects"""

    if metric==1: #Sum of Squared Distance Error (SSD)
        error = BlendingResult-S2
        #error = sum(error(:).^2)
    elif metric==2: #Mean Squared Error (MSE)
        error = np.sum((BlendingResult.astype("float") - S2.astype("float")) ** 2)
        error /= float(BlendingResult.shape[0] * BlendingResult.shape[1])
    return error

In [9]:
extracted = ExtractObject(s2, mask)

In [10]:
s1 = cv2.imread("Images/1_colour.jpeg", 1) #Scene with 1 object
blended = ObjectBlender(s1, extracted)

In [11]:
CompareResult(blended, s2, 2)

20031.10287109375

# Stage 2

In [36]:
def RemoveGreen(img):
    """Removes the green background from an image"""

    imgcop = np.copy(img)
    imgcop = cv2.cvtColor(imgcop, cv2.COLOR_BGR2RGB)

    lower_bound = np.array([31, 74, 40]) #Lower bound of green colour - darkest green
    upper_bound = np.array([136, 199, 179])

    greenscreen_mask = cv2.inRange(imgcop, lower_bound, upper_bound)
    
    masked_image = np.copy(imgcop)
    masked_image[greenscreen_mask != 0] = [0, 0, 0]

    cv2.imshow("Suppost elephant biss", masked_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return masked_image

In [12]:
def NewBackground(imgNoBg, NewBackground):
    """Replaces the background of an image with `NewBackground`"""

    

In [37]:
no_green = RemoveGreen(s1)

In [ ]:
#https://medium.com/fnplus/blue-or-green-screen-effect-with-open-cv-chroma-keying-94d4a6ab2743
new_back = cv2.imread("empty_controlled_depth8.png")
new_back = cv2.cvtColor(new_back, cv2.COLOR_BGR2RGB)
new_image = NewBackground(no_green, new_back)